<a href="https://colab.research.google.com/github/giovannibaratta/GTSRB/blob/master/ModelBuildStep1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title Import e definizioni (Versione 5) { vertical-output: true, display-mode: "form" }
#@markdown Selezionare se si vuole caricare una versione specifica di tensorflow
forceVersion = False #@param {type:"boolean"} 
tfVersion = "1.13.1" #@param ["1.12.2", "1.13.1", "1.14.0rc1", "1.14.0", "2.0.0b1", "PRINT_AV_VERSION"] {allow-input: true}
#@markdown Selezionare se si è collegati ad un runtime locale (non quello di colab)
localRuntime = False #@param{type:"boolean"}
#@markdown Selezionare se si intende utilizzare le TPU come acceleratore (solo se il runtime è quello di colab)
useTPU = False #@param {type:"boolean"}

useTPU = False if localRuntime else useTPU

if forceVersion == True:
  if useTPU == True:
    !pip install -q tensorflow=={tfVersion}
  else:
    !pip install -q tensorflow-gpu=={tfVersion}

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import pprint
import math
import random as rn
#data aug
from tensorflow import keras 
# gestione directory
import shutil
import sys
# colorare output
!pip install -q colorama
from colorama import Style as ColoramaStyle
import re
from datetime import datetime

currentPath = !pwd
#@markdown Path da utilizzare nel caso di runtime remoto
colabPath = '/gdrive/My Drive/DatasetSegnaliStradali/GTSRB/' #@param{type:'string'} 
#@markdown Path da utilizzare nel caso di runtime locale
localPath =  "/Desktop/DL/" #@param{type:'string'} 
completeLocalPath = currentPath[0] + localPath
#@markdown Cartella da utilizzare all'interno del path per l'output dei risultati (la cartella deve essere già presente)
phase = "trainingPhase1" #@param ["trainingPhase1", "trainingPhaseCNN", "trainingPhaseResNet", "trainingPhaseInception", "trainingPhaseGroupModel"] {allow-input: true}
rootPath = completeLocalPath if localRuntime else colabPath

#@markdown Dimensioni delle immagini di training, validation e test
width = 48#@param{type:'integer'}
height = 48#@param{type:'integer'}
tmpPath = rootPath + 'tmp/'
testDir = rootPath + 'data/test' + str(width) + 'x' + str(height) + '/'

print('Versione TF : ', ColoramaStyle.BRIGHT, tf.__version__, ColoramaStyle.RESET_ALL,sep="")

if useTPU == False:
  if 'COLAB_TPU_ADDR' in os.environ:
    raise RuntimeError("Hai selezionato il runtime TPU. Cambia l'impostazioni del notebook")
  print("Acceleratore : GPU")
  print("Verifica device :",tf.test.gpu_device_name())
  tpu_address = ''
else:
  if 'COLAB_TPU_ADDR' not in os.environ:
    print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
  else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print("Acceleratore : TPU")
    print ('Verifica device : ', tpu_address)

if not localRuntime:
  from google.colab import drive
  from google.colab import files
  drive.mount('/gdrive')
    
#import script da gdrive
scriptPath = rootPath + "scripts/colab/"
sys.path.append(scriptPath)

from CommonUtils import *
from ModelBuilderUtils import *
from TrainingUtils import *
from TestUtils import *

if tfVersion()['MAJOR'] < 2:
  tf.logging.set_verbosity(tf.logging.ERROR)
resetSeed()

groupInfo = {
    0 : 'prohibitory',
    1 : 'danger',
    2 : 'mandatory',
    3 : 'others',
    4 : 'limitsEnd'
}

labelsInfo = {
    # labelNumber : (group,labelInsideGroup, description)
    0 : (0,0, '20 km/h'),
    1 : (0,1, '30 km/h'),
    2 : (0,2, '50 km/h'),
    3 : (0,3, '60 km/h'),
    4 : (0,4, '70 km/h'),
    5 : (0,5, '80 km/h'),
    6 : (4,0, '80 km/h end'),
    7 : (0,6, '100 km/h'),
    8 : (0,7, '120 km/h'),
    9 : (0,8, 'No overtaking'),
    10 : (0,9, 'No overtaking for tracks'),
    11 : (1,0, 'Crossroad with secondary way'),
    12 : (3,0, 'Main road'),
    13 : (3,1,  'Give way'),
    14 : (3,2, 'Stop'),
    15 : (0,10,  'Road up'),
    16 : (0,11, 'Road up for track'),
    17 : (3,3,  'Brock'),
    18 : (1,1, 'Other dangerous'),
    19 : (1,2, 'Turn left'),
    20 : (1,3, 'Turn right'),
    21 : (1,4, 'Winding road'),
    22 : (1,5, 'Hollow road'),
    23 : (1,6, 'Slippery road'),
    24 : (1,7, 'Narrowing road'),
    25 : (1,8, 'Roadwork'),
    26 : (1,9, 'Traffic light'),
    27 : (1,10, 'Pedestrian'),
    28 : (1,11, 'Children'),
    29 : (1,12, 'Bike'),
    30 : (1,13, 'Snow'),
    31 : (1,14, 'Deer'),
    32 : (4,1, 'End of the limits'),
    33 : (2,0, 'Only right'),
    34 : (2,1, 'Only left'),
    35 : (2,2, 'Only straight'),
    36 : (2,3, 'Only straight and right'),
    37 : (2,4, 'Only straight and left'),
    38 : (2,5, 'Take right'),
    39 : (2,6, 'Take left'),
    40 : (2,7, 'Circle crossroad'),
    41 : (4,2, 'End of overtaking limit'),
    42 : (4,3, 'End of overtaking limit for track')
}

lookUpTable=[
    # gruppo 0
    {
        0:0,
        1:1,
        2:2,
        3:3,
        4:4,
        5:5,
        6:7,
        7:8,
        8:9,
        9:10,
        10:15,
        11:16
    },
    # gruppo 1
    {
        0:11,
        1:18,
        2:19,
        3:20,
        4:21,
        5:22,
        6:23,
        7:24,
        8:25,
        9:26,
        10:27,
        11:28,
        12:29,
        13:30,
        14:31
    },
    # gruppo 2
    {
        0:33,
        1:34,
        2:35,
        3:36,
        4:37,
        5:38,
        6:39,
        7:40
    },
    # gruppo 3
    {
        0:12,
        1:13,
        2:14,
        3:17
    },
    #gruppo 4
    {
        0:6,
        1:32,
        2:41,
        3:42  
    }
]

In [0]:
#@title Caricamento dei dati (versione 5) { vertical-output: true, display-mode: "form" }
if 'trainingData' in locals():
  del trainingData
if 'trainingLabels' in locals():
  del trainingLabels
if 'validationData' in locals():  
  del validationData
if 'validationLabels' in locals():
  del validationLabels

trainingDir = rootPath + 'data/training/'
validationDir = rootPath + 'data/validation/'

#@markdown Deselezionare nel caso si sti allenando il modello a due step, in quel caso le label non sono le originale ma sono il gruppo della classe
loadTrueLabels = True #@param{type:'boolean'}

samplingToLoad = 1#@param{type:'integer'}
#@markdown Seed per la genezione di nuove immagini
randomSeed = 62806#@param{type:'integer'}
#@markdown Sperimentale, non usare
use16Bit = False#@param{type:'boolean'}
#@markdown Sperimentale, non usare
convertToTesnor = False#@param{type:'boolean'}
trainingDir = rootPath + 'data/sampling' + str(samplingToLoad) + '_' + str(width) + 'x' + str(height) + '/training/'
validationDir = rootPath + 'data/sampling' + str(samplingToLoad) + '_' + str(width) + 'x' + str(height) + '/validation/'

#riproducibilità dei training
resetSeed(15 + randomSeed)

#@markdown Selezionare se si vogliono generare delle immagini aggiuntive
useDataAugmentation = False #@param {type:"boolean"}
#@markdown Se selezionato il valore degli slider vengono aggiunti ai valori presenti nel codice
usePerLabelDataAug = False #@param {type:"boolean"}
#@markdown Se selezionato non genera immagini aggiuntive per il validation set
onlyTrainingAug = False #@param {type:"boolean"}

#@markdown Numero di immagini da generare per ogni immagine originale
NUMBER_OF_TRAINING_AUG_GLOBAL = 0 #@param {type:"slider", min:0, max:20, step:1}
NUMBER_OF_VALIDATION_AUG_GLOBAL = 0 #@param {type:"slider", min:0, max:5, step:1}

# da modificare se si vuole alterare la distribuzione delle classi
NUMBER_AUG_BY_LABEL = { 
                0 : ( 4 , 2 ), # group 0  label inside group 0
                1 : ( 1 , 1 ), # group 0  label inside group 1
                2 : ( 1 , 1 ), # group 0  label inside group 2
                3 : ( 1 , 1 ), # group 0  label inside group 3
                4 : ( 1 , 1 ), # group 0  label inside group 4
                5 : ( 1 , 1 ), # group 0  label inside group 5
                6 : ( 3 , 2 ), # group 4  label inside group 0
                7 : ( 1 , 1 ), # group 0  label inside group 6
                8 : ( 1 , 1 ), # group 0  label inside group 7
                9 : ( 1 , 1 ), # group 0  label inside group 8
                10 : ( 1 , 1 ), # group 0  label inside group 9
                11 : ( 1 , 1 ), # group 1  label inside group 0
                12 : ( 1 , 1 ), # group 3  label inside group 0
                13 : ( 1 , 1 ), # group 3  label inside group 1
                14 : ( 2 , 1 ), # group 3  label inside group 2
                15 : ( 2 , 1 ), # group 0  label inside group 10
                16 : ( 3 , 2 ), # group 0  label inside group 11
                17 : ( 1 , 1 ), # group 3  label inside group 3
                18 : ( 1 , 1 ), # group 1  label inside group 1
                19 : ( 4 , 2 ), # group 1  label inside group 2
                20 : ( 3 , 2 ), # group 1  label inside group 3
                21 : ( 3 , 2 ), # group 1  label inside group 4
                22 : ( 3 , 2 ), # group 1  label inside group 5
                23 : ( 3 , 2 ), # group 1  label inside group 6
                24 : ( 3 , 2 ), # group 1  label inside group 7
                25 : ( 1 , 1 ), # group 1  label inside group 8
                26 : ( 3 , 2 ), # group 1  label inside group 9
                27 : ( 3 , 2 ), # group 1  label inside group 10
                28 : ( 3 , 2 ), # group 1  label inside group 11
                29 : ( 3 , 2 ), # group 1  label inside group 12
                30 : ( 4 , 2 ), # group 1  label inside group 13
                31 : ( 2 , 2 ), # group 1  label inside group 14
                32 : ( 3 , 2 ), # group 4  label inside group 1
                33 : ( 3 , 2 ), # group 2  label inside group 0
                34 : ( 5 , 2 ), # group 2  label inside group 1
                35 : ( 1 , 2 ), # group 2  label inside group 2
                36 : ( 2 , 2 ), # group 2  label inside group 3
                37 : ( 2 , 2 ), # group 2  label inside group 4
                38 : ( 1 , 1 ), # group 2  label inside group 5
                39 : ( 3 , 2 ), # group 2  label inside group 6
                40 : ( 3, 2 ), # group 2  label inside group 7
                41 : ( 3 , 2 ), # group 4  label inside group 2
                42 : ( 3 , 2 ), # group 4  label inside group 3
              }

NUMBER_OF_TRAINING_AUG = [0 for i in range(43)]
NUMBER_OF_VALIDATION_AUG = [0 for i in range(43)]

if useDataAugmentation == True:
  for key in labelsInfo.keys():
    NUMBER_OF_TRAINING_AUG[key] = NUMBER_AUG_BY_LABEL[key][0] if usePerLabelDataAug else NUMBER_OF_TRAINING_AUG_GLOBAL
    NUMBER_OF_VALIDATION_AUG[key] = NUMBER_AUG_BY_LABEL[key][1] if usePerLabelDataAug else NUMBER_OF_VALIDATION_AUG_GLOBAL
    if usePerLabelDataAug and onlyTrainingAug:
        NUMBER_OF_VALIDATION_AUG[key] = 0


brightness_range_training = (0.25,0.6)
rotation_range_training = 25
width_shift_range_training = 8
height_shift_range_training = 8
shear_range_training = 15
zoom_range_training =(0.4,1.15)
channel_shift_range_training = 0.13

brightness_range_validation =(0.25,0.7)
rotation_range_validation = 25
width_shift_range_validation = 8 
height_shift_range_validation = 8
shear_range_validation = 0.4
zoom_range_validation = (0.4,1.15)
channel_shift_range_validation = 0.12

#generatore utilizzato per le immagni di training
generatorTraining = keras.preprocessing.image.ImageDataGenerator(
    brightness_range = brightness_range_training,
    rotation_range= rotation_range_training,
    width_shift_range= width_shift_range_training,
    height_shift_range= height_shift_range_training,
    shear_range=shear_range_training,
    zoom_range= zoom_range_training,
    channel_shift_range=channel_shift_range_training)

#generatore utilizzato per le immagini di validation
generatorValidation = keras.preprocessing.image.ImageDataGenerator(
    brightness_range = brightness_range_validation,
    rotation_range=rotation_range_validation,
    width_shift_range=width_shift_range_validation,
    height_shift_range=height_shift_range_validation,
    shear_range=shear_range_validation,
    zoom_range=zoom_range_validation,
    channel_shift_range=channel_shift_range_validation)

# numero di immagini originali disponibili per ogni classe
trainingClassCount = {}
# numero di immagini di training totali
totalTrainingCount = 0

files = list(filter(lambda fn : fn.startswith('resized'), os.listdir(trainingDir)))
for fileName in files:
  classLabel = int(fileName.replace("resized",""))
  numFilePath = trainingDir + 'num' + str(classLabel)
  with open(numFilePath, 'r') as file:
    originalNum = int(file.readline())
    trainingClassCount[classLabel] = originalNum 
    totalTrainingCount += originalNum * (NUMBER_OF_TRAINING_AUG[classLabel] + 1)

print("Training images (no pad):", totalTrainingCount)

# se uso le TPU in tf 1.13 il numero di immagini deve essere divisibile per 8
trainingPadEnd = False

if useTPU == True and tfVersion()['MAJOR'] == 1 and tfVersion()['MAJOR'] <= 13:
  if totalTrainingCount % 8 != 0:
    trainingToPad = 8 - (totalTrainingCount % 8)
    totalTrainingCount = totalTrainingCount + (8 - (totalTrainingCount % 8))
    trainingPadEnd = True
  
#preparo gli array che contengono le img di training
trainingData = np.empty((totalTrainingCount, width, height, 3), dtype='float32')
trainingLabels = np.empty((totalTrainingCount), dtype='int32')
trueTrainingLabels = np.empty((totalTrainingCount), dtype='int32')

# numero di immagini originali disponibili per ogni classe
validationClassCount = {}
# numero di immagini di validation totali
totalValidationCount = 0

# recupero il numero di immagini di validation disponibili per ogni classe
files = list(filter(lambda fn : fn.startswith('resized'), os.listdir(trainingDir)))
for fileName in files:
  classLabel = int(fileName.replace("resized",""))
  numFilePath = validationDir + 'num' + str(classLabel)
  with open(numFilePath, 'r') as file:
    originalNum = int(file.readline())
    validationClassCount[classLabel] = originalNum
    totalValidationCount += originalNum * (NUMBER_OF_VALIDATION_AUG[classLabel] + 1)

print("Validation images (no pad):", totalValidationCount)

# se uso le TPU in tf 1.13 il numero di immagini deve essere divisibile per 8
validationPadEnd = False 

if useTPU == True and tfVersion()['MAJOR'] == 1 and tfVersion()['MAJOR'] <= 13:
    if totalValidationCount % 8 != 0:
      # se uso le TPU la lunghezza dei dati deve essere divisibile per 8
      validationToPad = 8 - (totalValidationCount % 8)
      totalValidationCount = totalValidationCount + (8 - (totalValidationCount % 8))
      validationPadEnd = True
  
#preparo gli array che contengono le img di validation
validationData = np.empty((totalValidationCount, width, height, 3), dtype='float32')
# contengono il gruppo da 0 a 4
validationLabels = np.empty((totalValidationCount), dtype='int32')
#contengono la label vera da 0 a 42
trueValidationLabels = np.empty((totalValidationCount), dtype='int32')

print('Total', totalValidationCount + totalTrainingCount)
if useDataAugmentation == True:
  print("Data augmentation training", NUMBER_OF_TRAINING_AUG, "\nvalidation", NUMBER_OF_VALIDATION_AUG)
else:
  print("Data augmentation : OFF")

#numero di immagini elaborate  
validationCount = 0
trainingCount = 0
loadedClass = 0

#caricamento dati
for fileName in files:
  classLabel = int(fileName.replace("resized",""))
  #carico le img di training originali
  trainingImages = np.memmap(trainingDir + fileName, dtype=np.uint8,
              mode='r', shape=(trainingClassCount[classLabel],width,height,3))

  for img in trainingImages:
    trainingData[trainingCount]  = img/255.0
    trainingLabels[trainingCount] = labelsInfo[classLabel][0] if not loadTrueLabels else classLabel
    trueTrainingLabels[trainingCount] = classLabel
    trainingCount += 1

  #carico le immagini di validation
  validationImages = np.memmap(validationDir + fileName, dtype=np.uint8,
              mode='r', shape=(validationClassCount[classLabel],width,height,3))

  for img in validationImages:
    validationData[validationCount] = img/255.0
    validationLabels[validationCount] = labelsInfo[classLabel][0] if not loadTrueLabels else classLabel
    trueValidationLabels[validationCount] = classLabel
    validationCount += 1
  
# per ogni immagine genero delle nuove immagini con trasformazioni casuali
if useDataAugmentation == True:
  print("Genero le immagini aggiuntive")
  resetSeed(48560  + randomSeed)
  for imageIndex in range(0, trainingCount):
    for augIndex in range(0, NUMBER_OF_TRAINING_AUG[trueTrainingLabels[imageIndex]]):
      trainingData[trainingCount] = generateRandomImage(generatorTraining, trainingData[imageIndex])/255.0
      trainingLabels[trainingCount] = trainingLabels[imageIndex]
      trueTrainingLabels[trainingCount] = trueTrainingLabels[imageIndex]
      trainingCount += 1

  resetSeed(-420360  + randomSeed)
  for imageIndex in range(0, validationCount):
    for augIndex in range(0, NUMBER_OF_VALIDATION_AUG[trueValidationLabels[imageIndex]]):
      validationData[validationCount] = generateRandomImage(generatorValidation, validationData[imageIndex])/255.0
      validationLabels[validationCount] = validationLabels[imageIndex]
      trueValidationLabels[validationCount] = trueValidationLabels[imageIndex]
      validationCount += 1

# Se necessario copio l'ultima immagine per rendere la lunghezza divisibile per 8
if trainingPadEnd == True:
  for index in range(1, trainingToPad + 1):
    trainingData[-index] = trainingData[len(trainingData) - (trainingToPad + 1)]
    trainingLabels[-index] = trainingLabels[len(trainingLabels) - (trainingToPad + 1)]
if validationPadEnd == True:
  for index in range(1, validationToPad + 1):
    validationData[-index] = validationData[len(validationData) - (validationToPad + 1)]
    validationLabels[-index] = validationLabels[len(validationLabels) - (validationToPad + 1)]

print("")

if use16Bit:
    trainingData = tf.convert_to_tensor(trainingData, dtype=tf.bfloat16)
    validationData = tf.convert_to_tensor(validationData, dtype=tf.bfloat16)
else:
    if convertToTesnor:
        tempTrainingData = tf.convert_to_tensor(trainingData[:1000], dtype=tf.float32)
        tempValidationData = tf.convert_to_tensor(validationData[:1000], dtype=tf.float32)

        for index in range((len(trainingData)//1000)-1):
            tempTrainingData = tf.concat([tempTrainingData, tf.convert_to_tensor(trainingData[1000+1000*index:1000+1000*(index+1)], dtype=tf.float32)],0)
        for index in range((len(validationData)//1000)-1):
            tempValidationData = tf.concat([tempValidationData, tf.convert_to_tensor(validationData[1000+1000*index:1000+1000*(index+1)], dtype=tf.float32)],0)
        trainingData = tf.concat([tempTrainingData, tf.convert_to_tensor(trainingData[1000*(len(trainingData)//1000):len(trainingData)], dtype=tf.float32)],0)
        validationData = tf.concat([tempValidationData, tf.convert_to_tensor(validationData[1000*(len(validationData)//1000):len(validationData)], dtype=tf.float32)],0)
print('Training', trainingData.shape) 
print('Validation', validationData.shape)

# statistiche sulla distribuzione delle labels
plt.figure(figsize=(20,5))
maxLabel = max(trainingLabels)+1
print(maxLabel)
if maxLabel > 43:
  raise RuntimeError("C'è un errore grave nel codice, ricontrolla.")
plt.hist(trainingLabels, alpha = 0.5, bins=maxLabel, label = "training")
plt.hist(validationLabels, alpha = 0.5, bins=maxLabel, label = "validation")
plt.xticks(np.arange(0, maxLabel, 1))
plt.xlim(left=0, right=maxLabel)
plt.legend(loc='upper right')
plt.show()

In [0]:
def phase1_1D64():
  def closure():
    inputs = tf.keras.layers.Input(shape=(width, height, 3), name = "L0_INPUT")
    layer = buildDenseLayer(inputs, layers = 1, size = 64, regularizers = 0.01, flattenInput = True)
    outputs = buildDenseSoftmax(layer)
    return tf.keras.Model(inputs = inputs, outputs = outputs)
  return (closure, "phase1_1D64")

def phase1_2D64():
  def closure():
    inputs = tf.keras.layers.Input(shape=(width, height, 3), name = "L0_INPUT")
    layer = buildDenseLayer(inputs, layers = 2, size = 64, regularizers = 0.01, dropout = 0.3,flattenInput = True)
    outputs = buildDenseSoftmax(layer)
    return tf.keras.Model(inputs = inputs, outputs = outputs)
  return (closure, "phase1_2D64")

def phase1_1D128():
  def closure():
    inputs = tf.keras.layers.Input(shape=(width, height, 3), name = "L0_INPUT")
    layer = buildDenseLayer(inputs, layers = 1, size = 128, regularizers = 0.01,flattenInput = True)
    outputs = buildDenseSoftmax(layer)
    return tf.keras.Model(inputs = inputs, outputs = outputs)
  return (closure, "phase1_1D128")

def phase1_2D128():
  def closure():
    inputs = tf.keras.layers.Input(shape=(width, height, 3), name = "L0_INPUT")
    layer = buildDenseLayer(inputs, layers = 2, size = 128, regularizers = 0.01, dropout = 0.3,flattenInput = True)
    outputs = buildDenseSoftmax(layer)
    return tf.keras.Model(inputs = inputs, outputs = outputs)
  return (closure, "phase1_2D128")

def phase1_1C64_3():
  def closure():
    inputs = tf.keras.layers.Input(shape=(width, height, 3), name = "L0_INPUT")
    layer = buildConvLayer(inputs, layers = 1, size = 64, kernelSize = 3, flatten = True)
    outputs = buildDenseSoftmax(layer)
    return tf.keras.Model(inputs = inputs, outputs = outputs)
  return (closure, "phase1_1C64_3")

def phase1_2C64_3():
  def closure():
    inputs = tf.keras.layers.Input(shape=(width, height, 3), name = "L0_INPUT")
    layer = buildConvLayer(inputs, layers = 2, size = 64, kernelSize = 3, flatten = True)
    outputs = buildDenseSoftmax(layer)
    return tf.keras.Model(inputs = inputs, outputs = outputs)
  return (closure, "phase1_2C64_3")

def phase1_2C64_3_1D64():
  def closure():
    inputs = tf.keras.layers.Input(shape=(width, height, 3), name = "L0_INPUT")
    layer = buildConvLayer(inputs, layers = 2, size = 64, kernelSize = 3, flatten = True)
    layer =  buildDenseLayer(layer, layers = 1, size = 64, regularizers = 0.01, flattenInput = False)
    outputs = buildDenseSoftmax(layer)
    return tf.keras.Model(inputs = inputs, outputs = outputs)
  return (closure, "phase1_2C64_3_1D64")

def phase1_2C64_3_1MAX2_2_1D64():
  def closure():
    inputs = tf.keras.layers.Input(shape=(width, height, 3), name = "L0_INPUT")
    layer = buildConvBlock(inputs, layers = 2, size = 64, kernelSize = 3, poolSize = 2, poolStrides = 2, flatten = False)
    layer =  buildDenseLayer(layer, layers = 1, size = 64, regularizers = 0.01, flattenInput = True)
    outputs = buildDenseSoftmax(layer)
    return tf.keras.Model(inputs = inputs, outputs = outputs)
  return (closure, "phase1_2C64_3_1MAX2_2_1D64")

def phase1_3C64_3_1I_G():
  def closure():
    inputs = tf.keras.layers.Input(shape=(width, height, 3), name = "L0_INPUT")
    layer = buildConvBlock(inputs, layers = 3, size = 64, kernelSize = 3, poolSize = 2, poolStrides = 2, flatten = False)
    layer = buildInceptionBlock(layer)
    outputs = buildGlobalSoftmax(layer)
    return tf.keras.Model(inputs = inputs, outputs = outputs)
  return (closure, "phase1_3C64_3_1I_G")

def phase1_3C64_3_1R_G():
  def closure():
    inputs = tf.keras.layers.Input(shape=(width, height, 3), name = "L0_INPUT")
    layer = buildConvBlock(inputs, layers = 3, size = 64, kernelSize = 3, poolSize = 2, poolStrides = 2, flatten = False)
    layer = buildResNetBlock(layer, 128, 128, 512)
    outputs = buildGlobalSoftmax(layer)
    return tf.keras.Model(inputs = inputs, outputs = outputs)
  return (closure, "phase1_3C64_3_1R_G")

In [0]:
models = Models()

trInfo = TrainingInfo.getDefaultTPU(
    trainingData,
    trainingLabels,
    validationData,
    validationLabels
)

trInfo.setParameters(
    learningRateList = [1e-2,1e-3,1e-4,1e-5,1e-6,1e-7],
    fineTuningIterations = 0,
    mainEpochs = 50,
    batchSize = 128,
    validationFrequency = 1,
    metrics = ['sparse_categorical_accuracy'],
    classWeights = None
)

generatedModel = phase1_2C64_3_1D64()
models.addModel(generatedModel[1], generatedModel[0], trInfo)

In [0]:
#@title Training ( versione 4) { vertical-output: true, display-mode: "form" }
cleanLastSession = False #@param {type:"boolean"}
if 'lastSession' in locals():
  if cleanLastSession == True:
    for trainedModel in lastSession:
      print("Rimuovo il training del modello", trainedModel)
      shutil.rmtree(trainedModel)
    lastSession = []
else:
  lastSession = []
#@markdown ---
verboseTraining = 0 #@param {type:"integer"}
#@markdown ---
mainTrainingEarlyStoppingDelta = 0.01 #@param {type:"number"}
mainTrainingEarlyStoppinPatience = 15 #@param {type:"integer"}
fineTuningEarlyStoppingDelta = 0.003 #@param {type:"number"}
fineTuningEarlyStoppinPatience = 10 #@param {type:"integer"}
#@markdown ---

trainingPath = rootPath + phase + "/"

dataToLog  = ["AUG : "+ str(useDataAugmentation)+"\n",
                    "TR_AUG : " + str(NUMBER_OF_TRAINING_AUG)+"\n",
                    "VAL_AUG : " + str(NUMBER_OF_VALIDATION_AUG)+"\n",
                    "brightness_range_training : " + str(brightness_range_training) + "\n",
                    "rotation_range_training : " + str(rotation_range_training) + "\n",
                    "width_shift_range_training : " + str(width_shift_range_training) + "\n",
                    "height_shift_range_training : " + str(height_shift_range_training) + "\n",
                    "shear_range_training : " + str(shear_range_training) + "\n",     
                    "zoom_range_training : " + str(zoom_range_training) + "\n",
                    "channel_shift_range_training : " + str(channel_shift_range_training) + "\n",
                    "brightness_range_validation : " + str(brightness_range_validation) + "\n",
                    "rotation_range_validation : " + str(rotation_range_validation) + "\n",
                    "width_shift_range_validation : " + str(width_shift_range_validation) + "\n",
                    "height_shift_range_validation : " + str(height_shift_range_validation) + "\n",
                    "shear_range_validation : " + str(shear_range_validation) + "\n",     
                    "zoom_range_validation : " + str(zoom_range_validation) + "\n",
                    "channel_shift_range_validation : " + str(channel_shift_range_validation) + "\n",
                    "sampling : " + str(samplingToLoad) + "\n",
                    "randomSeed : " + str(randomSeed)+"\n"]

if "useCustomLoad" in locals() and useCustomLoad == True:
  dataToLog.append("Custom_load : " + str(useCustomLoad)+"\n")
  dataToLog.append("Custom_ratio : " + str(customRatio) + "\n")  
        
train(tpu_address,
  trainingPath,
  models, 
  verboseTraining = 1,
  mainTrainingEarlyStoppingDelta = mainTrainingEarlyStoppingDelta,
  mainTrainingEarlyStoppinPatience = mainTrainingEarlyStoppinPatience,
  fineTuningEarlyStoppingDelta = fineTuningEarlyStoppingDelta,
  fineTuningEarlyStoppinPatience = fineTuningEarlyStoppinPatience,
  stringToLog = ''.join(dataToLog),
  lastSession = lastSession)
  
    
print("Fine training")